# Interpolation Methods in RustLab-Numerical

This notebook demonstrates the comprehensive interpolation capabilities of rustlab-numerical, showcasing 1D and 2D interpolation methods with practical examples and performance analysis.

## Setup and Dependencies

Import the required crates and modules for interpolation, mathematical operations, and plotting.

In [2]:
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-numerical = { path = ".." }
:dep rustlab-plotting = { path = "../../rustlab-plotting" }

use rustlab_math::{VectorF64, ArrayF64, vec64, array64, range, FunctionalMap};
use rustlab_numerical::interpolation::*;
use rustlab_plotting::*;
use std::f64::consts::PI;

## 1. Linear Interpolation

Linear interpolation is the simplest and fastest method, providing O(log n) evaluation time.

In [3]:
{
    // Create sample data points: quadratic function y = x²
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    
    // Create linear interpolator using the correct API
    let linear_interp = LinearInterpolator::new(x_data.clone(), y_data.clone())?;
    
    // Evaluate interpolation at specific points
    println!("Linear Interpolation Results:");
    println!("f(0.5) = {:.3}", linear_interp.eval(0.5)?);
    println!("f(1.5) = {:.3}", linear_interp.eval(1.5)?);
    println!("f(2.5) = {:.3}", linear_interp.eval(2.5)?);
    
    // Check domain
    let (min_x, max_x) = linear_interp.domain();
    println!("Domain: [{:.1}, {:.1}]", min_x, max_x);
}

Linear Interpolation Results:
f(0.5) = 0.500
f(1.5) = 2.500
f(2.5) = 6.500
Domain: [0.0, 5.0]


()

## 2. Cubic Spline Interpolation

Cubic splines provide smooth C² continuous interpolation with minimal curvature.

In [4]:
{
    // Create the same data points
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    
    // Create natural cubic spline using the correct API
    let spline = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural)?;
    
    // Evaluate spline at specific points
    println!("Cubic Spline Interpolation Results:");
    println!("f(0.5) = {:.3}", spline.eval(0.5)?);
    println!("f(1.5) = {:.3}", spline.eval(1.5)?);
    println!("f(2.5) = {:.3}", spline.eval(2.5)?);
    
    // Evaluate derivatives (splines support derivatives)
    println!("\nDerivative Information:");
    println!("f'(1.5) = {:.3}", spline.eval_derivative(1.5)?);
    println!("f''(1.5) = {:.3}", spline.eval_second_derivative(1.5)?);
}

Cubic Spline Interpolation Results:
f(0.5) = 0.342
f(1.5) = 2.224
f(2.5) = 6.263

Derivative Information:
f'(1.5) = 3.026
f''(1.5) = 2.211


()

## 3. Comparison of Interpolation Methods

Visualize how different interpolation methods compare for the same dataset.

In [5]:
{
    // Create data points
    let x_data = vec64![0.0, 1.0, 2.0, 3.0, 4.0, 5.0];
    let y_data = vec64![0.0, 1.0, 4.0, 9.0, 16.0, 25.0];
    
    // Create interpolators
    let linear_interp = LinearInterpolator::new(x_data.clone(), y_data.clone())?;
    let spline = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural)?;
    
    // Create evaluation points using range macro
    let x_eval = range!(0.0 => 5.0, 101);
    
    // Evaluate both interpolators using eval_vec method
    let y_linear = linear_interp.eval_vec(&x_eval)?;
    let y_spline = spline.eval_vec(&x_eval)?;
    
    // Create exact values for comparison using ergonomic map
    let y_exact = x_eval.map(|x| x * x);

    // Create comparison plot using correct plotting API
    Plot::new()
        .scatter_with(&x_data, &y_data, "Data Points")
        .line_with(&x_eval, &y_exact, "Exact (x²)")
        .line_with(&x_eval, &y_linear, "Linear")
        .line_with(&x_eval, &y_spline, "Cubic Spline")
        .title("Interpolation Method Comparison")
        .xlabel("x")
        .ylabel("y")
        .legend(true)
        .grid(true)
        .show()?;
    
    println!("Comparison plot generated successfully!");
}

Comparison plot generated successfully!


()

## 4. Polynomial Interpolation

Demonstrate Lagrange and Newton polynomial interpolation methods.

In [6]:
{
    // Create smaller dataset for polynomial interpolation
    let x_data = vec64![0.0, 1.0, 2.0, 3.0];
    let y_data = vec64![1.0, 2.0, 5.0, 10.0];
    
    // Create polynomial interpolators
    let lagrange = LagrangeInterpolator::new(x_data.clone(), y_data.clone())?;
    let newton = NewtonInterpolator::new(x_data.clone(), y_data.clone())?;
    
    // Evaluate at test points
    let test_points = vec![0.5, 1.5, 2.5];
    
    println!("Polynomial Interpolation Comparison:");
    println!("{:<8} {:<12} {:<12}", "x", "Lagrange", "Newton");
    
    for &x in &test_points {
        let lag_val = lagrange.eval(x)?;
        let newt_val = newton.eval(x)?;
        println!("{:<8.1} {:<12.4} {:<12.4}", x, lag_val, newt_val);
    }
    
    // Both methods should give identical results
    let diff = (lagrange.eval(1.5)? - newton.eval(1.5)?).abs();
    println!("\nDifference between methods: {:.2e}", diff);
}

Polynomial Interpolation Comparison:
x        Lagrange     Newton      
0.5      1.2500       1.2500      
1.5      3.2500       3.2500      
2.5      7.2500       7.2500      

Difference between methods: 0.00e0


()

## 5. Spline Boundary Conditions

Compare different boundary conditions for cubic splines.

In [7]:
{
    // Create data for sine function using ergonomic math operations
    let x_data = vec64![0.0, PI/4.0, PI/2.0, 3.0*PI/4.0, PI];
    let y_data = x_data.sin(); // Ergonomic vectorized sine operation!
    
    // Create splines with different boundary conditions
    let natural = CubicSpline::new(x_data.clone(), y_data.clone(), BoundaryCondition::Natural)?;
    let clamped = CubicSpline::new(
        x_data.clone(), 
        y_data.clone(), 
        BoundaryCondition::Clamped { 
            start_derivative: 1.0,  // cos(0) = 1
            end_derivative: -1.0    // cos(π) = -1
        }
    )?;
    
    // Evaluate at boundary to show difference
    println!("Boundary Condition Comparison (derivatives at x=0):");
    println!("Natural spline f'(0) = {:.3}", natural.eval_derivative(0.0)?);
    println!("Clamped spline f'(0) = {:.3}", clamped.eval_derivative(0.0)?);
    println!("Exact cos(0) = {:.3}", 1.0);
    
    println!("\nBoundary Condition Comparison (derivatives at x=π):");
    println!("Natural spline f'(π) = {:.3}", natural.eval_derivative(PI)?);
    println!("Clamped spline f'(π) = {:.3}", clamped.eval_derivative(PI)?);
    println!("Exact cos(π) = {:.3}", -1.0);
}

Boundary Condition Comparison (derivatives at x=0):
Natural spline f'(0) = 0.998
Clamped spline f'(0) = 1.000
Exact cos(0) = 1.000

Boundary Condition Comparison (derivatives at x=π):
Natural spline f'(π) = -0.998
Clamped spline f'(π) = -1.000
Exact cos(π) = -1.000


()

## 6. Extrapolation Modes

Demonstrate different extrapolation behaviors when evaluating outside the domain.

In [8]:
{
    // Create simple linear data
    let x_data = vec64![1.0, 2.0, 3.0];
    let y_data = vec64![2.0, 4.0, 6.0];  // y = 2x
    
    // Test different extrapolation modes
    let linear_error = LinearInterpolator::new(x_data.clone(), y_data.clone())?;
    let linear_const = LinearInterpolator::new(x_data.clone(), y_data.clone())?
        .with_extrapolation(ExtrapolationMode::Constant);
    let linear_linear = LinearInterpolator::new(x_data.clone(), y_data.clone())?
        .with_extrapolation(ExtrapolationMode::Linear);
    let linear_nan = LinearInterpolator::new(x_data.clone(), y_data.clone())?
        .with_extrapolation(ExtrapolationMode::NaN);
    
    let test_x = 0.5;  // Outside domain [1, 3]
    
    println!("Extrapolation at x = {} (outside domain [1, 3]):", test_x);
    
    // Error mode
    match linear_error.eval(test_x) {
        Ok(val) => println!("Error mode: {:.3}", val),
        Err(_) => println!("Error mode: Returns error (as expected)"),
    }
    
    // Constant mode
    println!("Constant mode: {:.3}", linear_const.eval(test_x)?);
    
    // Linear mode
    println!("Linear mode: {:.3}", linear_linear.eval(test_x)?);
    
    // NaN mode
    let nan_result = linear_nan.eval(test_x)?;
    let nan_display = if nan_result.is_nan() { 
        "NaN".to_string() 
    } else { 
        format!("{:.3}", nan_result) 
    };
    println!("NaN mode: {}", nan_display);
    
    println!("\nFor comparison, exact y = 2*{} = {:.1}", test_x, 2.0 * test_x);
}

Extrapolation at x = 0.5 (outside domain [1, 3]):
Error mode: Returns error (as expected)
Constant mode: 2.000
Linear mode: 1.000
NaN mode: NaN

For comparison, exact y = 2*0.5 = 1.0


()

## 7. 2D Bilinear Interpolation

Demonstrate interpolation on 2D grids using bilinear interpolation.

In [9]:
{
    // Create 2D grid data
    let x_grid = vec64![0.0, 1.0, 2.0];
    let y_grid = vec64![0.0, 1.0, 2.0];
    
    // Create surface data: z = x + y
    let mut z_data = array64![[0.0, 1.0, 2.0],   // y=0: z = x
                              [1.0, 2.0, 3.0],   // y=1: z = x+1  
                              [2.0, 3.0, 4.0]];  // y=2: z = x+2
    
    // Create bilinear interpolator (remove references)
    let bilinear = BilinearInterpolator::new(x_grid.clone(), y_grid.clone(), z_data.clone())?;
    
    // Test interpolation at various points
    let test_points = [(0.5, 0.5), (1.0, 0.5), (0.5, 1.0), (1.5, 1.5)];
    
    println!("2D Bilinear Interpolation Results:");
    println!("{:<12} {:<12} {:<12} {:<12}", "x", "y", "Interpolated", "Exact (x+y)");
    
    for &(x, y) in &test_points {
        let interp_val = bilinear.eval(x, y)?;
        let exact_val = x + y;
        println!("{:<12.1} {:<12.1} {:<12.3} {:<12.1}", x, y, interp_val, exact_val);
    }
    
    // Show domain
    let ((x_min, x_max), (y_min, y_max)) = bilinear.domain();
    println!("\n2D Domain: x ∈ [{:.1}, {:.1}], y ∈ [{:.1}, {:.1}]", 
             x_min, x_max, y_min, y_max);
}

2D Bilinear Interpolation Results:
x            y            Interpolated Exact (x+y) 
0.5          0.5          1.000        1.0         
1.0          0.5          1.500        1.5         
0.5          1.0          1.500        1.5         
1.5          1.5          3.000        3.0         

2D Domain: x ∈ [0.0, 2.0], y ∈ [0.0, 2.0]


()

## 8. Signal Reconstruction Example

Practical example: reconstruct a signal from sparse samples.

In [10]:
{
    // Create a complex signal using ergonomic vectorized operations
    let t_full = range!(0.0 => 4.0 * PI, 200);
    let signal_full = (&t_full).sin() + (&t_full * 3.0).sin() * 0.5 + (&t_full * 10.0).cos() * 0.1;

    // Subsample the signal (simulate missing data)
    let step = 8;
    let t_sparse_vec: Vec<f64> = t_full.iter().step_by(step).copied().collect();
    let signal_sparse_vec: Vec<f64> = signal_full.iter().step_by(step).copied().collect();
    let t_sparse = VectorF64::from_slice(&t_sparse_vec);
    let signal_sparse = VectorF64::from_slice(&signal_sparse_vec);

    println!("Original signal: {} samples", t_full.len());
    println!("Sparse signal: {} samples ({:.1}% of original)", 
             t_sparse.len(), 
             100.0 * t_sparse.len() as f64 / t_full.len() as f64);

    // Reconstruct using different methods
    let linear_signal = LinearInterpolator::new(t_sparse.clone(), signal_sparse.clone())?;
    let spline_signal = CubicSpline::new(t_sparse.clone(), signal_sparse.clone(), BoundaryCondition::Natural)?;

    // Get the interpolation domain to ensure evaluation stays within bounds
    let (t_min, t_max) = linear_signal.domain();
    println!("Interpolation domain: [{:.3}, {:.3}]", t_min, t_max);
    
    // Create evaluation points within the interpolation domain
    let t_eval = range!(t_min => t_max, 200);
    let signal_eval_exact = (&t_eval).sin() + (&t_eval * 3.0).sin() * 0.5 + (&t_eval * 10.0).cos() * 0.1;

    // Evaluate reconstructed signals within domain
    let signal_linear = linear_signal.eval_vec(&t_eval)?;
    let signal_spline = spline_signal.eval_vec(&t_eval)?;

    // Plot comparison
    Plot::new()
        .line_with(&t_eval, &signal_eval_exact, "Original")
        .scatter_with(&t_sparse, &signal_sparse, "Sparse Samples")
        .line_with(&t_eval, &signal_linear, "Linear Reconstruction")
        .line_with(&t_eval, &signal_spline, "Spline Reconstruction")
        .title("Signal Reconstruction from Sparse Samples")
        .xlabel("Time")
        .ylabel("Amplitude")
        .legend(true)
        .grid(true)
        .show()?;

    println!("Signal reconstruction plot generated!");
}

Original signal: 200 samples
Sparse signal: 25 samples (12.5% of original)
Interpolation domain: [0.000, 12.124]
Signal reconstruction plot generated!


()

## 9. Performance Benchmarking

Compare the computational performance of different interpolation methods.

In [11]:
{
    use std::time::Instant;

    // Create larger dataset for benchmarking using ergonomic operations
    let n_points = 1000;
    let x_large = range!(0.0 => 10.0, n_points);
    let y_large = (&x_large * PI).sin(); // Use reference to avoid move

    // Create interpolators
    let start = Instant::now();
    let linear_large = LinearInterpolator::new(x_large.clone(), y_large.clone())?;
    let linear_setup_time = start.elapsed();

    let start = Instant::now();
    let spline_large = CubicSpline::new(x_large.clone(), y_large.clone(), BoundaryCondition::Natural)?;
    let spline_setup_time = start.elapsed();

    // Benchmark evaluation - make sure to stay within domain [0, 10]
    let n_evals = 10000;
    let eval_points = range!(0.1 => 9.9, n_evals);

    // Linear interpolation benchmark
    let start = Instant::now();
    let _linear_results = linear_large.eval_vec(&eval_points)?;
    let linear_eval_time = start.elapsed();

    // Spline interpolation benchmark
    let start = Instant::now();
    let _spline_results = spline_large.eval_vec(&eval_points)?;
    let spline_eval_time = start.elapsed();

    println!("Performance Benchmark Results:");
    println!("Dataset size: {} points", n_points);
    println!("Evaluation points: {}", n_evals);
    println!();
    println!("Linear interpolation setup: {:?}", linear_setup_time);
    println!("Linear interpolation eval:  {:?} ({:.1} ns/point)", 
             linear_eval_time, 
             linear_eval_time.as_nanos() as f64 / n_evals as f64);
    println!();
    println!("Spline interpolation setup: {:?}", spline_setup_time);
    println!("Spline interpolation eval:  {:?} ({:.1} ns/point)", 
             spline_eval_time,
             spline_eval_time.as_nanos() as f64 / n_evals as f64);
    println!();
    println!("Spline setup overhead: {:.1}x", 
             spline_setup_time.as_nanos() as f64 / linear_setup_time.as_nanos() as f64);
    println!("Spline eval overhead: {:.1}x", 
             spline_eval_time.as_nanos() as f64 / linear_eval_time.as_nanos() as f64);
}

Performance Benchmark Results:
Dataset size: 1000 points
Evaluation points: 10000

Linear interpolation setup: 6.496µs
Linear interpolation eval:  632.199µs (63.2 ns/point)

Spline interpolation setup: 54.43µs
Spline interpolation eval:  432.784µs (43.3 ns/point)

Spline setup overhead: 8.4x
Spline eval overhead: 0.7x


()

## 10. Error Analysis

Analyze interpolation errors for different methods on a known function.

In [12]:
{
    // Test function: f(x) = exp(-x²) (Gaussian) using ergonomic operations
    let x_coarse = vec64![-2.0, -1.0, 0.0, 1.0, 2.0];
    let y_coarse = ((&x_coarse).map(|x| x * x) * (-1.0)).exp(); // Ergonomic: (-x²).exp()
    
    // Create interpolators
    let linear = LinearInterpolator::new(x_coarse.clone(), y_coarse.clone())?;
    let spline = CubicSpline::new(x_coarse.clone(), y_coarse.clone(), BoundaryCondition::Natural)?;
    
    // Fine evaluation grid for error analysis
    let x_fine = range!(-2.0 => 2.0, 201);
    
    // Compute exact values using ergonomic operations
    let y_exact = ((&x_fine).map(|x| x * x) * (-1.0)).exp();
    
    // Compute interpolated values
    let y_linear = linear.eval_vec(&x_fine)?;
    let y_spline = spline.eval_vec(&x_fine)?;
    
    // Calculate errors with small epsilon to avoid log(0)
    let epsilon = 1e-16;
    let mut linear_errors = Vec::new();
    let mut spline_errors = Vec::new();
    
    for i in 0..x_fine.len() {
        let exact = y_exact.get(i).unwrap();
        let lin = y_linear.get(i).unwrap();
        let spl = y_spline.get(i).unwrap();
        
        // Add epsilon to avoid log(0) issues in plotting
        linear_errors.push((lin - exact).abs() + epsilon);
        spline_errors.push((spl - exact).abs() + epsilon);
    }
    
    let max_linear_error = linear_errors.iter().fold(0.0_f64, |a, &b| a.max(b));
    let max_spline_error = spline_errors.iter().fold(0.0_f64, |a, &b| a.max(b));
    let avg_linear_error = linear_errors.iter().sum::<f64>() / linear_errors.len() as f64;
    let avg_spline_error = spline_errors.iter().sum::<f64>() / spline_errors.len() as f64;
    
    println!("Error Analysis for Gaussian function exp(-x²):");
    println!("Sampling points: {}", x_coarse.len());
    println!("Evaluation points: {}", x_fine.len());
    println!();
    println!("Linear interpolation max error:  {:.2e}", max_linear_error);
    println!("Linear interpolation avg error:  {:.2e}", avg_linear_error);
    println!();
    println!("Spline interpolation max error:  {:.2e}", max_spline_error);
    println!("Spline interpolation avg error:  {:.2e}", avg_spline_error);
    println!();
    println!("Spline improvement (max): {:.1}x", max_linear_error / max_spline_error);
    println!("Spline improvement (avg): {:.1}x", avg_linear_error / avg_spline_error);
    
    // Plot error comparison
    let linear_err_vec = VectorF64::from_slice(&linear_errors);
    let spline_err_vec = VectorF64::from_slice(&spline_errors);
    
    Plot::new()
        .line_with(&x_fine, &linear_err_vec, "Linear Error")
        .line_with(&x_fine, &spline_err_vec, "Spline Error")
        .title("Interpolation Error Analysis")
        .xlabel("x")
        .ylabel("Absolute Error")
        .yscale(Scale::Log10)
        .legend(true)
        .grid(true)
        .show()?;
    
    println!("Error analysis plot generated!");
}

Error Analysis for Gaussian function exp(-x²):
Sampling points: 5
Evaluation points: 201

Linear interpolation max error:  1.06e-1
Linear interpolation avg error:  6.00e-2

Spline interpolation max error:  2.38e-2
Spline interpolation avg error:  1.01e-2

Spline improvement (max): 4.5x
Spline improvement (avg): 5.9x
Error analysis plot generated!


()

## Summary

This notebook demonstrated the comprehensive interpolation capabilities of rustlab-numerical:

### Key Takeaways:

1. **Linear Interpolation**: Fast and simple, ideal for quick approximations
2. **Cubic Splines**: Smooth interpolation with derivative support, best for most applications
3. **Polynomial Methods**: Exact interpolation but can suffer from Runge phenomenon
4. **Boundary Conditions**: Affect spline behavior at domain edges
5. **Extrapolation Modes**: Control behavior outside interpolation domain
6. **2D Interpolation**: Bilinear and bicubic methods for surface data
7. **Performance**: Linear is fastest, splines offer best accuracy/performance trade-off
8. **Error Analysis**: Splines typically provide significantly better accuracy

### Best Practices:

- Use **linear interpolation** for real-time applications and simple data
- Use **cubic splines** for smooth functions requiring derivative information
- Use **polynomial interpolation** sparingly and only for small datasets
- Choose appropriate **boundary conditions** based on known function behavior
- Consider **extrapolation modes** carefully to avoid unexpected behavior
- **Benchmark performance** for your specific use case and dataset size

The rustlab-numerical interpolation module provides a robust, efficient, and mathematically sound foundation for interpolation tasks in scientific computing applications.